# Transfer Learning em Python (Colab): Cats vs Dogs (ou seu próprio dataset)

Este notebook realiza **Transfer Learning** com TensorFlow/Keras usando **MobileNetV2** como base (pré-treinada no ImageNet).
Você pode rodar com o dataset **Cats vs Dogs (TFDS)** ou adaptar para **sua própria base** (duas classes).

> Repositório DIO: depois de finalizar, suba este notebook e o `README.md` para seu GitHub.


In [ ]:
#@title 🔧 Checagem de ambiente (GPU) e versões
import tensorflow as tf, sys, platform, os

print("TF:", tf.__version__)
print("Python:", sys.version.split()[0])
print("OS:", platform.platform())
print("GPU disponível:", tf.config.list_physical_devices('GPU'))


In [ ]:
#@title 🔒 Reprodutibilidade (definir sementes)
import random, numpy as np, tensorflow as tf
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


## 1) Escolha a fonte dos dados

Você tem **duas opções**:

1. **TFDS (recomendado):** baixa automaticamente `cats_vs_dogs`.
2. **Pasta no Drive / upload próprio:** duas pastas, uma por classe, por ex.:
```
dataset/
 ├─ cats/
 └─ dogs/
```


In [ ]:
#@title 1A) Usar TFDS: Cats vs Dogs (automático)
# Essa opção não exige download manual (apenas internet ativa no Colab).
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras

IMG_SIZE = 224
BATCH_SIZE = 32

(ds_train_full, ds_test_raw), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:90%]", "train[90%:]"],  # TFDS não traz test pronto; separamos parte final como "teste"
    with_info=True,
    as_supervised=True,
)

# Criar validação a partir do treino
train_size = 0.8
ds_size = ds_info.splits["train"].num_examples * 0.9  # pois usamos 90% como "treino inteiro"
val_take = int((1 - train_size) * ds_size)

ds_val = ds_train_full.take(val_take)
ds_train = ds_train_full.skip(val_take)

class_names = ["cat", "dog"]
num_classes = 2

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)                   .shuffle(2048, seed=SEED)                   .batch(BATCH_SIZE)                   .prefetch(AUTOTUNE)

ds_val = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE)               .batch(BATCH_SIZE)               .prefetch(AUTOTUNE)

ds_test = ds_test_raw.map(preprocess, num_parallel_calls=AUTOTUNE)                     .batch(BATCH_SIZE)                     .prefetch(AUTOTUNE)

print("Batches (train/val/test):", len(ds_train), len(ds_val), len(ds_test))


In [ ]:
#@title 1B) Usar seu próprio dataset (pasta no Drive) { run: "false" }
# Descomente e ajuste o caminho do diretório com subpastas por classe.
# Dica: monte o Drive no Colab: 
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_DIR = "/content/drive/MyDrive/datasets/cats_dogs"  # ajuste aqui!

# import tensorflow as tf
# from tensorflow import keras
# IMG_SIZE = 224
# BATCH_SIZE = 32
# seed = SEED

# train_ds = keras.preprocessing.image_dataset_from_directory(
#     DATA_DIR,
#     validation_split=0.2,
#     subset="training",
#     seed=seed,
#     image_size=(IMG_SIZE, IMG_SIZE),
#     batch_size=BATCH_SIZE,
# )
# val_ds = keras.preprocessing.image_dataset_from_directory(
#     DATA_DIR,
#     validation_split=0.2,
#     subset="validation",
#     seed=seed,
#     image_size=(IMG_SIZE, IMG_SIZE),
#     batch_size=BATCH_SIZE,
# )
# class_names = train_ds.class_names
# num_classes = len(class_names)

# AUTOTUNE = tf.data.AUTOTUNE
# ds_train = train_ds.prefetch(AUTOTUNE)
# ds_val   = val_ds.prefetch(AUTOTUNE)

# # Opcional: criar um pequeno conjunto de teste a partir da validação
# ds_test = ds_val


In [ ]:
#@title 2) Data Augmentation (recomendado para generalização)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="augmentation")


In [ ]:
#@title 3) Construção do modelo (Transfer Learning com MobileNetV2)
from tensorflow.keras import layers, models

IMG_SIZE = 224
base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base.trainable = False  # Fase 1: congelado

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()


In [ ]:
#@title 4) Treinamento - Fase 1 (base congelada)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

EPOCHS_1 = 5

ckpt_path = "model_phase1.keras"
callbacks = [
    EarlyStopping(patience=2, restore_best_weights=True, monitor="val_accuracy"),
    ModelCheckpoint(ckpt_path, save_best_only=True, monitor="val_accuracy")
]

history1 = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=EPOCHS_1,
    callbacks=callbacks
)


In [ ]:
#@title 5) Fine-tuning - Fase 2 (descongelar parte da base)
# Descongelar últimas camadas para refinar pesos pré-treinados.
base.trainable = True
# "Descongela" a partir de um certo índice de camada para evitar overfitting/estabilidade
fine_tune_at = len(base.layers) - 30
for layer in base.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

EPOCHS_2 = 5
ckpt_path2 = "model_phase2.keras"
callbacks2 = [
    EarlyStopping(patience=2, restore_best_weights=True, monitor="val_accuracy"),
    ModelCheckpoint(ckpt_path2, save_best_only=True, monitor="val_accuracy")
]

history2 = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=EPOCHS_2,
    callbacks=callbacks2
)


In [ ]:
#@title 6) Curvas de treino (accuracy e loss)
import matplotlib.pyplot as plt

def plot_history(h, title_prefix="Phase"):
    acc = h.history.get("accuracy", [])
    val_acc = h.history.get("val_accuracy", [])
    loss = h.history.get("loss", [])
    val_loss = h.history.get("val_loss", [])

    # Accuracy
    plt.figure()
    plt.plot(acc, label="train_acc")
    plt.plot(val_acc, label="val_acc")
    plt.title(f"{title_prefix} - Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

    # Loss
    plt.figure()
    plt.plot(loss, label="train_loss")
    plt.plot(val_loss, label="val_loss")
    plt.title(f"{title_prefix} - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

plot_history(history1, "Phase 1")
plot_history(history2, "Phase 2")


In [ ]:
#@title 7) Avaliação no conjunto de teste
test_loss, test_acc = model.evaluate(ds_test)
print("Test accuracy:", round(test_acc, 4), "— loss:", round(test_loss, 4))


In [ ]:
#@title 8) Matriz de confusão e exemplos de previsão
import numpy as np
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf

y_true = []
y_pred = []

for images, labels in ds_test:
    preds = model.predict(images, verbose=0)
    y_true.extend(labels.numpy().tolist())
    y_pred.extend(np.argmax(preds, axis=1).tolist())

from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_true, y_pred)

def plot_confusion_matrix(cm, classes):
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(cm, class_names)
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
#@title 9) Salvar modelo (Keras e TensorFlow Lite)
# Salvar em formato Keras
model.save("cats_dogs_transfer.keras")

# Exportar para SavedModel
import tensorflow as tf
tf.saved_model.save(model, "export_savedmodel")

# Converter para TFLite (opcional)
converter = tf.lite.TFLiteConverter.from_saved_model("export_savedmodel")
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("Arquivos gerados: cats_dogs_transfer.keras, export_savedmodel/, model.tflite")


In [ ]:
#@title 10) (Opcional) Salvar no Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# !mkdir -p "/content/drive/MyDrive/transfer_learning"
# !cp -r cats_dogs_transfer.keras export_savedmodel model.tflite "/content/drive/MyDrive/transfer_learning/"


## Como adaptar para **duas outras classes** (seu projeto)

- Troque a fonte dos dados (seção 1B) apontando para a pasta com **duas classes** (nomes das subpastas = rótulos).
- `num_classes` será definido automaticamente a partir de `class_names` com `image_dataset_from_directory`.
- Para base diferente, você pode substituir `MobileNetV2` por `EfficientNetB0`, `ResNet50`, etc.
- Ajuste `fine_tune_at`, `EPOCHS_1`, `EPOCHS_2` e `learning_rate` conforme necessário.


## Referências úteis
- TFDS Cats vs Dogs: https://www.tensorflow.org/datasets/catalog/cats_vs_dogs  
- Dataset (Microsoft Research download): https://www.microsoft.com/en-us/download/details.aspx?id=54765  
- Exemplo de Transfer Learning (ml4a): https://colab.research.google.com/github/kylemath/ml4a-guides/blob/master/notebooks/transfer-learning.ipynb
